# Cloud Pak for Data Virtualization Demonstration
This notebook will connect to a CP4D cluster and demonstrate how SQL can be run against a variety of data sources without requiring any knowledge of how these data sources are accessed.

In [ ]:
%run db2.ipynb
%run connection.ipynb

## Stock Symbol Table
### Get information about the stocks that are in the database
**System Z - VSAM**

In [ ]:
%sql -grid SELECT SYMBOL, COMPANY FROM DVDEMO.STOCK_SYMBOLS

## Top Buy/Sell By Customer
**AWS - Db2 on OpenShift, Azure - Db2 on OpenShift, Azure - Postgres**

In [ ]:
%%sql -grid
WITH
    TX_DAY(TX_DATE) AS (
      VALUES NEXT_DAY(DATE('2018-01-01') + INT(RAND()*350) DAYS, 'Thursday')
    ),
    PURCHASED(CUSTID,AMOUNT) AS (
      SELECT CUSTID, SUM(QUANTITY) AS AMOUNT
        FROM FOLDING.STOCK_TRANSACTIONS_DV ST, TX_DAY TX
      WHERE ST.TX_DATE = TX.TX_DATE 
      GROUP BY CUSTID
    )
    SELECT C.LASTNAME, C.CITY, C.STATE, P.AMOUNT
      FROM PURCHASED P, TRADING.CUSTOMERS C
      WHERE C.CUSTID = P.CUSTID
    ORDER BY ABS(P.AMOUNT) DESC
    FETCH FIRST 9 ROWS ONLY 

## Customers
**VMware - MongoDB**

In [ ]:
%%sql -grid
SELECT * FROM TRADING.CUSTOMERS